# Overall Stats

In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 

In [2]:
dfDeliveries = pd.read_csv("../data/IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv("../data/IPL_Matches_2008_2022.csv")


Merge the dataframes

In [3]:
dfMerged = dfDeliveries.merge(dfMatches, on='ID')

In [10]:
dfMerged.head(5)

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,...,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2,BowlingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,...,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon,Gujarat Titans


In [9]:
dfMerged.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'extra_type', 'batsman_run', 'extras_run', 'total_run',
       'non_boundary', 'isWicketDelivery', 'player_out', 'kind',
       'fielders_involved', 'BattingTeam', 'City', 'Date', 'Season',
       'MatchNumber', 'Team1', 'Team2', 'Venue', 'TossWinner', 'TossDecision',
       'SuperOver', 'WinningTeam', 'WonBy', 'Margin', 'method',
       'Player_of_Match', 'Team1Players', 'Team2Players', 'Umpire1', 'Umpire2',
       'BowlingTeam'],
      dtype='object')

# Cleaning Team Names

In [135]:
dfMerged['WinningTeam'].value_counts()

WinningTeam
Mumbai Indians                 31490
Chennai Super Kings            29221
Kolkata Knight Riders          26839
Royal Challengers Bangalore    25673
Rajasthan Royals               23132
Kings XI Punjab                20948
Sunrisers Hyderabad            17770
Delhi Daredevils               15709
Delhi Capitals                  8754
Deccan Chargers                 7013
Punjab Kings                    3142
Gujarat Lions                   3063
Gujarat Titans                  2920
Pune Warriors                   2883
Rising Pune Supergiant          2383
Lucknow Super Giants            2215
Kochi Tuskers Kerala            1322
Rising Pune Supergiants         1105
Name: count, dtype: int64

In [139]:
dfMerged['WinningTeam']=dfMerged['WinningTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['WinningTeam']=dfMerged['WinningTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['WinningTeam']=dfMerged['WinningTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [140]:
dfMerged['WinningTeam'].value_counts()

WinningTeam
Mumbai Indians                 31490
Chennai Super Kings            29221
Kolkata Knight Riders          26839
Royal Challengers Bangalore    25673
Delhi Capitals                 24463
Punjab Kings                   24090
Rajasthan Royals               23132
Sunrisers Hyderabad            17770
Deccan Chargers                 7013
Rising Pune Supergiant          3488
Gujarat Lions                   3063
Gujarat Titans                  2920
Pune Warriors                   2883
Lucknow Super Giants            2215
Kochi Tuskers Kerala            1322
Name: count, dtype: int64

In [143]:
dfMerged['Team1'].value_counts()

Team1
Royal Challengers Bangalore    29522
Mumbai Indians                 26971
Chennai Super Kings            26909
Delhi Capitals                 26874
Punjab Kings                   26133
Kolkata Knight Riders          24733
Rajasthan Royals               21234
Sunrisers Hyderabad            16374
Deccan Chargers                 9448
Pune Warriors                   5483
Gujarat Lions                   3784
Rising Pune Supergiant          3294
Lucknow Super Giants            1962
Gujarat Titans                  1670
Kochi Tuskers Kerala            1563
Name: count, dtype: int64

In [142]:
dfMerged['Team1']=dfMerged['Team1'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['Team1']=dfMerged['Team1'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['Team1']=dfMerged['Team1'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [146]:
dfMerged['Team2'].value_counts()

Team2
Mumbai Indians                 28709
Kolkata Knight Riders          27814
Delhi Capitals                 25954
Punjab Kings                   25676
Rajasthan Royals               24408
Royal Challengers Bangalore    23702
Chennai Super Kings            23070
Sunrisers Hyderabad            19961
Deccan Chargers                 8625
Pune Warriors                   5417
Rising Pune Supergiant          3729
Gujarat Lions                   3327
Gujarat Titans                  2208
Lucknow Super Giants            1721
Kochi Tuskers Kerala            1633
Name: count, dtype: int64

In [145]:
dfMerged['Team2']=dfMerged['Team2'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['Team2']=dfMerged['Team2'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['Team2']=dfMerged['Team2'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [149]:
dfMerged['TossWinner'].value_counts()

TossWinner
Mumbai Indians                 29521
Delhi Capitals                 27269
Kolkata Knight Riders          26989
Chennai Super Kings            26208
Royal Challengers Bangalore    24826
Rajasthan Royals               23565
Punjab Kings                   22512
Sunrisers Hyderabad            17490
Deccan Chargers                10376
Pune Warriors                   4798
Gujarat Lions                   3586
Rising Pune Supergiant          2874
Gujarat Titans                  2418
Lucknow Super Giants            1762
Kochi Tuskers Kerala            1760
Name: count, dtype: int64

In [148]:
dfMerged['TossWinner']=dfMerged['TossWinner'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['TossWinner']=dfMerged['TossWinner'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['TossWinner']=dfMerged['TossWinner'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [152]:
dfMerged['BattingTeam'].value_counts()

BattingTeam
Mumbai Indians                 27826
Royal Challengers Bangalore    26512
Delhi Capitals                 26373
Kolkata Knight Riders          26192
Punjab Kings                   26034
Chennai Super Kings            25128
Rajasthan Royals               22777
Sunrisers Hyderabad            18196
Deccan Chargers                 9034
Pune Warriors                   5443
Gujarat Lions                   3566
Rising Pune Supergiant          3480
Gujarat Titans                  1971
Lucknow Super Giants            1840
Kochi Tuskers Kerala            1582
Name: count, dtype: int64

In [151]:
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

Adding the bowling team column

In [153]:
dfMerged['BowlingTeam'] = np.where(dfMerged['BattingTeam']==dfMerged['Team1'],
                                       dfMerged['Team2'],
                                       dfMerged['Team1'])

Cleaning the season column

In [13]:
dfMerged['Season'].unique()

array(['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
       '2014', '2013', '2012', '2011', '2010', '2009', '2008'],
      dtype=object)

In [12]:
dfMerged['Season'] = dfMerged['Season'].replace({'2020/21': '2020', '2009/10': '2010', '2007/08': '2008'})

batting group and bowling group

In [154]:
batgroup = dfMerged.groupby(['BattingTeam'])
bowlgroup = dfMerged.groupby(['BowlingTeam'])

batting Stats collection

In [155]:
batting_team_stats = pd.DataFrame(batgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_faced'})

batting_team_stats['matches_played'] = batgroup['ID'].nunique()

batting_team_stats['runs'] = batgroup['batsman_run'].sum()

batting_team_stats['4s'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['4s'].fillna(0,inplace=True)

batting_team_stats['6s'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['6s'].fillna(0,inplace=True)

batting_team_stats['1s'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['1s'].fillna(0,inplace=True)

batting_team_stats['2s'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['2s'].fillna(0,inplace=True)

batting_team_stats['0s'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
batting_team_stats['0s'].fillna(0,inplace=True)

batting_team_stats['dismissals'] = batgroup['isWicketDelivery'].sum()


In [92]:
batting_team_stats.head(5)

,balls_faced,matches_played,runs,4s,6s,1s,2s,0s,dismissals
BattingTeam,,,,,,,,,
Chennai Super Kings,25128,208,31734,2788,1269,9399,1623,8663,1089
Deccan Chargers,9034,75,10885,957,400,3387,563,3227,484
Delhi Capitals,7587,63,9691,912,347,2956,458,2519,372
Delhi Daredevils,18786,161,23022,2156,801,7011,1176,6595,952
Gujarat Lions,3566,30,4629,460,155,1360,233,1155,188


# bowling stats

In [156]:
bowling_team_stats = pd.DataFrame(bowlgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_bowled'})



bowling_team_stats['runs_conceded'] = batgroup['total_run'].sum()

bowling_team_stats['4s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['4s_conceded'].fillna(0,inplace=True)

bowling_team_stats['6s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['6s_conceded'].fillna(0,inplace=True)

bowling_team_stats['1s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['1s_conceded'].fillna(0,inplace=True)

bowling_team_stats['2s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['2s_conceded'].fillna(0,inplace=True)

bowling_team_stats['0s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
bowling_team_stats['0s_conceded'].fillna(0,inplace=True)

bowling_team_stats['wickets_taken'] = bowlgroup['isWicketDelivery'].sum()

In [60]:
dfMerged['extra_type'].unique()

array([nan, 'legbyes', 'wides', 'byes', 'noballs', 'penalty'],
      dtype=object)

In [157]:
bowling_team_stats['noballs_conceded'] = dfMerged[dfMerged['extra_type'] == 'noballs'].groupby('BowlingTeam')['total_run'].count()
bowling_team_stats['noballs_conceded'].fillna(0,inplace=True)
bowling_team_stats['legbyes_conceded'] = dfMerged[dfMerged['extra_type'] == 'legbyes'].groupby('BowlingTeam')['total_run'].count()
bowling_team_stats['legbyes_conceded'].fillna(0,inplace=True)
bowling_team_stats['wides_conceded'] = dfMerged[dfMerged['extra_type'] == 'wides'].groupby('BowlingTeam')['total_run'].count()
bowling_team_stats['wides_conceded'].fillna(0,inplace=True)
bowling_team_stats['byes_conceded'] = dfMerged[dfMerged['extra_type'] == 'byes'].groupby('BowlingTeam')['total_run'].count()
bowling_team_stats['byes_conceded'].fillna(0,inplace=True)

In [69]:
dfMerged['kind'].unique()

array([nan, 'caught', 'caught and bowled', 'run out', 'bowled', 'stumped',
       'lbw', 'hit wicket', 'retired hurt', 'retired out',
       'obstructing the field'], dtype=object)

In [158]:
bowling_team_stats['catches_taken'] = dfMerged[dfMerged['kind'] == 'caught'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['catches_taken'].fillna(0,inplace= True)
bowling_team_stats['run_out_taken'] = dfMerged[dfMerged['kind'] == 'run out'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['run_out_taken'].fillna(0,inplace= True)
bowling_team_stats['bowled_taken'] = dfMerged[dfMerged['kind'] == 'bowled'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['bowled_taken'].fillna(0,inplace= True)
bowling_team_stats['stumped_taken'] = dfMerged[dfMerged['kind'] == 'stumped'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['stumped_taken'].fillna(0,inplace= True)
bowling_team_stats['lbw_taken'] = dfMerged[dfMerged['kind'] == 'lbw'].groupby('BowlingTeam')['isWicketDelivery'].count()
bowling_team_stats['lbw_taken'].fillna(0,inplace= True)

In [96]:
bowling_team_stats.head(5)

,balls_bowled,runs_conceded,4s_conceded,6s_conceded,1s_conceded,2s_conceded,0s_conceded,wickets_taken,noballs_conceded,catches_taken,run_out_taken,bowled_taken,stumped_taken,lbw_taken
BowlingTeam,,,,,,,,,,,,,,
Chennai Super Kings,24851,33393,2714,1157,9168,1603,8848,1290,81,823,109,197,34,83
Deccan Chargers,9039,11463,1013,405,3181,579,3306,446,49,271,38,82,17,26
Delhi Capitals,7730,10145,847,396,3049,432,2627,405,29,275,22,63,13,21
Delhi Daredevils,18725,24296,2180,870,6953,1139,6521,912,86,528,116,173,30,44
Gujarat Lions,3545,4862,453,190,1354,285,1097,151,7,94,18,21,3,11


combining batting and bowling stats

In [159]:
combined_team_stats = pd.merge(batting_team_stats, bowling_team_stats, left_index=True, right_index=True,
                               how='outer')
combined_team_stats.reset_index(inplace=True)
combined_team_stats.rename(columns={'BattingTeam': 'Team'}, inplace=True)

combined_team_stats


,Team,balls_faced,matches_played,runs,4s,6s,1s,2s,0s,dismissals,...,wickets_taken,noballs_conceded,legbyes_conceded,wides_conceded,byes_conceded,catches_taken,run_out_taken,bowled_taken,stumped_taken,lbw_taken
0,Chennai Super Kings,25128,208,31734,2788,1269,9399,1623,8663,1089,...,1290,81,371,782,49,823,109,197,34,83
1,Deccan Chargers,9034,75,10885,957,400,3387,563,3227,484,...,446,49,166,280,31,271,38,82,17,26
2,Delhi Capitals,26373,224,32713,3068,1148,9967,1634,9114,1324,...,1317,115,416,779,53,803,138,236,43,65
3,Gujarat Lions,3566,30,4629,460,155,1360,233,1155,188,...,151,7,55,91,7,94,18,21,3,11
4,Gujarat Titans,1971,16,2517,250,79,794,121,622,82,...,101,6,22,55,3,64,10,12,3,9
5,Kochi Tuskers Kerala,1582,14,1758,170,53,570,85,595,86,...,74,7,29,52,2,35,12,16,1,8
6,Kolkata Knight Riders,26192,223,32330,3021,1227,9545,1507,9378,1318,...,1275,114,405,733,87,722,123,244,50,96
7,Lucknow Super Giants,1840,15,2405,188,115,711,118,595,89,...,100,6,27,72,2,71,7,15,1,4
8,Mumbai Indians,27826,231,34703,3156,1410,10030,1607,9980,1383,...,1411,120,434,923,72,857,118,250,38,89
9,Pune Warriors,5443,45,6040,525,196,1992,351,2099,298,...,238,26,71,120,18,154,18,39,7,16


In [160]:
combined_team_stats.shape

(15, 27)

In [161]:
combined_team_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Team              15 non-null     object
 1   balls_faced       15 non-null     int64 
 2   matches_played    15 non-null     int64 
 3   runs              15 non-null     int64 
 4   4s                15 non-null     int64 
 5   6s                15 non-null     int64 
 6   1s                15 non-null     int64 
 7   2s                15 non-null     int64 
 8   0s                15 non-null     int64 
 9   dismissals        15 non-null     int64 
 10  balls_bowled      15 non-null     int64 
 11  runs_conceded     15 non-null     int64 
 12  4s_conceded       15 non-null     int64 
 13  6s_conceded       15 non-null     int64 
 14  1s_conceded       15 non-null     int64 
 15  2s_conceded       15 non-null     int64 
 16  0s_conceded       15 non-null     int64 
 17  wickets_taken     

In [162]:
combined_team_stats.to_csv('../derived/combined_team_stats_overall.csv',index=False)

# per season stats

In [163]:
def calculate_season_stats(df,season):
    # Filter data for the desired season
    dfMerged = df[df['Season'] == season]
    
    batgroup = dfMerged.groupby(['BattingTeam'])
    bowlgroup = dfMerged.groupby(['BowlingTeam'])
    
    #batting stats collection
    batting_team_stats = pd.DataFrame(batgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_faced'})

    batting_team_stats['matches_played'] = batgroup['ID'].nunique()

    batting_team_stats['runs'] = batgroup['batsman_run'].sum()

    batting_team_stats['4s'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()
    batting_team_stats['4s'].fillna(0,inplace=True)

    batting_team_stats['6s'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BattingTeam')['batsman_run'].count()
    batting_team_stats['6s'].fillna(0,inplace=True)

    batting_team_stats['1s'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
    batting_team_stats['1s'].fillna(0,inplace=True)

    batting_team_stats['2s'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
    batting_team_stats['2s'].fillna(0,inplace=True)

    batting_team_stats['0s'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BattingTeam')['batsman_run'].count()
    batting_team_stats['0s'].fillna(0,inplace=True)

    batting_team_stats['dismissals'] = batgroup['isWicketDelivery'].sum()
    
    #bowling stats colleciton
    bowling_team_stats = pd.DataFrame(bowlgroup['ballnumber'].count()).rename(columns={'ballnumber':'balls_bowled'})



    bowling_team_stats['runs_conceded'] = batgroup['total_run'].sum()

    bowling_team_stats['4s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 4) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
    bowling_team_stats['4s_conceded'].fillna(0,inplace=True)

    bowling_team_stats['6s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 6) & (dfMerged['non_boundary'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
    bowling_team_stats['6s_conceded'].fillna(0,inplace=True)

    bowling_team_stats['1s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 1) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
    bowling_team_stats['1s_conceded'].fillna(0,inplace=True)

    bowling_team_stats['2s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 2) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
    bowling_team_stats['2s_conceded'].fillna(0,inplace=True)

    bowling_team_stats['0s_conceded'] = dfMerged[(dfMerged['batsman_run'] == 0) & (dfMerged['extras_run'] == 0)].groupby('BowlingTeam')['batsman_run'].count()
    bowling_team_stats['0s_conceded'].fillna(0,inplace=True)

    bowling_team_stats['wickets_taken'] = bowlgroup['isWicketDelivery'].sum()
    
    bowling_team_stats['noballs_conceded'] = dfMerged[dfMerged['extra_type'] == 'noballs'].groupby('BowlingTeam')['total_run'].count()
    bowling_team_stats['noballs_conceded'].fillna(0,inplace=True)
    bowling_team_stats['legbyes_conceded'] = dfMerged[dfMerged['extra_type'] == 'legbyes'].groupby('BowlingTeam')['total_run'].count()
    bowling_team_stats['legbyes_conceded'].fillna(0,inplace=True)
    bowling_team_stats['wides_conceded'] = dfMerged[dfMerged['extra_type'] == 'wides'].groupby('BowlingTeam')['total_run'].count()
    bowling_team_stats['wides_conceded'].fillna(0,inplace=True)
    bowling_team_stats['byes_conceded'] = dfMerged[dfMerged['extra_type'] == 'byes'].groupby('BowlingTeam')['total_run'].count()
    bowling_team_stats['byes_conceded'].fillna(0,inplace=True)
    
    bowling_team_stats['catches_taken'] = dfMerged[dfMerged['kind'] == 'caught'].groupby('BowlingTeam')['isWicketDelivery'].count()
    bowling_team_stats['catches_taken'].fillna(0,inplace= True)
    bowling_team_stats['run_out_taken'] = dfMerged[dfMerged['kind'] == 'run out'].groupby('BowlingTeam')['isWicketDelivery'].count()
    bowling_team_stats['run_out_taken'].fillna(0,inplace= True)
    bowling_team_stats['bowled_taken'] = dfMerged[dfMerged['kind'] == 'bowled'].groupby('BowlingTeam')['isWicketDelivery'].count()
    bowling_team_stats['bowled_taken'].fillna(0,inplace= True)
    bowling_team_stats['stumped_taken'] = dfMerged[dfMerged['kind'] == 'stumped'].groupby('BowlingTeam')['isWicketDelivery'].count()
    bowling_team_stats['stumped_taken'].fillna(0,inplace= True)
    bowling_team_stats['lbw_taken'] = dfMerged[dfMerged['kind'] == 'lbw'].groupby('BowlingTeam')['isWicketDelivery'].count()
    bowling_team_stats['lbw_taken'].fillna(0,inplace= True)
    
    combined_team_stats = pd.merge(batting_team_stats, bowling_team_stats, left_index=True, right_index=True,
                               how='outer')
    combined_team_stats.reset_index(inplace=True)
    combined_team_stats.rename(columns={'BattingTeam': 'Team'}, inplace=True)
    
    combined_team_stats['Season'] = season
    
    return combined_team_stats
    
    
    

In [164]:
calculate_season_stats(dfMerged,'2022')

,Team,balls_faced,matches_played,runs,4s,6s,1s,2s,0s,dismissals,...,noballs_conceded,legbyes_conceded,wides_conceded,byes_conceded,catches_taken,run_out_taken,bowled_taken,stumped_taken,lbw_taken,Season
0,Chennai Super Kings,1719,14,2165,183,103,614,90,626,88,...,6,22,75,3,58,4,12,0.0,7,2022
1,Delhi Capitals,1650,14,2218,205,106,572,82,584,81,...,7,18,65,2,64,2,11,4.0,5,2022
2,Gujarat Titans,1971,16,2517,250,79,794,121,622,82,...,6,22,55,3,64,10,12,3.0,9,2022
3,Kolkata Knight Riders,1634,14,2109,177,113,508,97,635,104,...,12,25,49,7,57,8,8,3.0,3,2022
4,Lucknow Super Giants,1840,15,2405,188,115,711,118,595,89,...,6,27,72,2,71,7,15,1.0,4,2022
5,Mumbai Indians,1691,14,2100,173,100,587,102,633,94,...,2,29,73,1,64,5,9,0.0,6,2022
6,Punjab Kings,1712,14,2193,200,110,556,79,634,99,...,9,19,66,6,65,6,7,2.0,1,2022
7,Rajasthan Royals,2107,17,2807,249,137,731,115,757,97,...,7,28,110,4,68,7,18,2.0,8,2022
8,Royal Challengers Bangalore,1965,16,2454,222,102,677,116,704,98,...,6,26,62,4,63,7,10,2.0,9,2022
9,Sunrisers Hyderabad,1623,14,2084,170,97,613,98,565,80,...,8,27,75,4,51,5,21,1.0,5,2022


getting all season stats

In [165]:
all_season_stats = []

# Assuming dfMerged is your main DataFrame
for season in dfMerged['Season'].unique():
    season_stats = calculate_season_stats(dfMerged, season)
    all_season_stats.append(season_stats)

# Concatenate the list of dataframes into a single dataframe
all_season_stats_df = pd.concat(all_season_stats, ignore_index=True)

# Now, all_season_stats_df contains the combined statistics for all seasons


# Done

In [131]:
dfMerged["Season"].unique()

array(['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
       '2014', '2013', '2012', '2011', '2010', '2009', '2008'],
      dtype=object)

In [166]:
all_season_stats_df['Team'].unique()

array(['Chennai Super Kings', 'Delhi Capitals', 'Gujarat Titans',
       'Kolkata Knight Riders', 'Lucknow Super Giants', 'Mumbai Indians',
       'Punjab Kings', 'Rajasthan Royals', 'Royal Challengers Bangalore',
       'Sunrisers Hyderabad', 'Gujarat Lions', 'Rising Pune Supergiant',
       'Pune Warriors', 'Deccan Chargers', 'Kochi Tuskers Kerala'],
      dtype=object)

In [168]:
all_season_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Team              126 non-null    object 
 1   balls_faced       126 non-null    int64  
 2   matches_played    126 non-null    int64  
 3   runs              126 non-null    int64  
 4   4s                126 non-null    int64  
 5   6s                126 non-null    int64  
 6   1s                126 non-null    int64  
 7   2s                126 non-null    int64  
 8   0s                126 non-null    int64  
 9   dismissals        126 non-null    int64  
 10  balls_bowled      126 non-null    int64  
 11  runs_conceded     126 non-null    int64  
 12  4s_conceded       126 non-null    int64  
 13  6s_conceded       126 non-null    int64  
 14  1s_conceded       126 non-null    int64  
 15  2s_conceded       126 non-null    int64  
 16  0s_conceded       126 non-null    int64  
 1

In [167]:
teams_per_season = all_season_stats_df.groupby('Season')['Team'].nunique()
print(teams_per_season)


Season
2008     8
2009     8
2010     8
2011    10
2012     9
2013     9
2014     8
2015     8
2016     8
2017     8
2018     8
2019     8
2020     8
2021     8
2022    10
Name: Team, dtype: int64


In [169]:
all_season_stats_df.to_csv('../derived/all_season_team_stats_overall.csv',index=False)